In [1]:
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics, new_dynamics2
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

In [2]:
length=[4, 4]
V = 1.0
h = 1.0
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)


ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))


hex_ = nk.machine.new_hex(np.array(length))

ma = nk.machine.RbmDimer(hi, hex_, alpha = 1, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)

yoyo
1


### set params to 0 (represent ground state for v=1)

In [3]:
ma._ws[:] = 0
ma._set_bare_parameters(
    ma._a, ma._b, ma._w, ma._as, ma._bs, ma._ws, ma._autom, ma._z2
)

### create Markov Chain 
number of markov chain is denoted by n_samples_

In [4]:
importlib.reload(nk)

<module 'netket' from '../netket/__init__.py'>

In [5]:
n_jobs = 6
length=[4, 4]
V = 1.0
h = 1.0

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op, ad2o_o, op_num, label_num  = f.dimer_flip1(length = np.array(length), return_info = True)
# op = f.dimer_flip_old(length = np.array(length))

ad2_bool = np.zeros([ad2o_o.shape[0], ad2o_o.shape[0]], dtype = np.bool)
for l in range(ad2o_o.shape[0]):
    label = ad2o_o[l]
    for op_ in label:
        ad2_bool[l,op_] = True
        
ma.hex.ad2o_o = ad2o_o.astype(np.int64)
ma.hex.ad2_bool = ad2_bool
hex_ = nk.machine.new_hex(np.array(length))




sweep_size = 1000
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op, length = length, n_chains=1, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sa_mul.reset()
_ = sa_mul.generate_samples(1000) # discard the begginings of metropolis sampling.

number of core : 6


In [6]:
n_samples_ = 500000
samples_state = sa_mul.generate_samples(int(n_samples_ / 1))
samples_state = samples_state.reshape(-1, ma.hilbert.size)
print('get samples')

get samples


In [8]:
import scripts.dynamics as Dynamics
from importlib import reload
reload(Dynamics)
t_list = np.arange(0,20,0.1)

# d = Dynamics.new_dynamics(op, ma)
# %time P_ = d.multiprocess(samples_state, 300, 12) 
# d = Dynamics.new_dynamics3(op, ma)
# %time P_ = d.multiprocess(samples_state, t_list, 12) 
import scripts.dynamics_4 as sdy
reload(sdy)


d = sdy.new_dynamics_one(op, ma)
P_list = d.multiprocess(samples_state, t_list,6) 

In [9]:
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))

In [10]:
from importlib import reload
reload(f)


operators = f.return_vison(hi, np.arange(hex_.lattice_coor.shape[0]))

In [11]:
num_samples = (P_list[:,:,0]!=0).sum(axis=1)
P_list_ = P_list.reshape(-1,P_list.shape[-1])
sections1 = np.arange(P_list.shape[1])
sections2 = np.zeros(P_list_.shape[0])

_, mels1, _ = operators[0].get_conn_flattened(P_list[0,:,:], sections1)
sub1 = mels1.mean().real
mels1 = mels1.real

size = len(operators)

dimer_corr = np.zeros((size,t_list.shape[0]))
dimer_std = np.zeros((size,t_list.shape[0]))


for s in range(1):
    print(s)
    
    if operators[s]:
        sub2 = operators[s].get_conn_flattened(P_list[0,:,:], sections1)[1].mean().real
        _, mels2_, _ = operators[s].get_conn_flattened(P_list_, sections2)
        mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real
        dimer_corr[s] = (mels2 * mels1).mean(axis=1)
        dimer_std[s] = (mels2 * mels1).std(axis=1)
    else:
        dimer_corr[s] = 0
        dimer_std[s] = 0

yo
0
yo
yo


In [12]:
dimer_corr[0]

array([1.        , 0.94125977, 0.89362357, 0.85278741, 0.81783927,
       0.78729115, 0.76026704, 0.73692295, 0.71514686, 0.69589878,
       0.67743471, 0.66146665, 0.64611058, 0.63219453, 0.61811047,
       0.60519842, 0.59276237, 0.58097432, 0.56969028, 0.55882624,
       0.5487902 , 0.53849815, 0.52839411, 0.51927008, 0.51025004,
       0.501006  , 0.49245797, 0.48422994, 0.47633791, 0.46896188,
       0.46154185, 0.45404582, 0.44764979, 0.44122576, 0.43472174,
       0.42782971, 0.42144969, 0.41512566, 0.40876564, 0.40232161,
       0.39615758, 0.38892956, 0.38249753, 0.37708151, 0.37254149,
       0.36564546, 0.36034144, 0.35602542, 0.3505574 , 0.34543738,
       0.34134537, 0.33643335, 0.33188533, 0.32740931, 0.32212529,
       0.31787727, 0.31297325, 0.30880124, 0.30365721, 0.2996892 ,
       0.29538118, 0.29040116, 0.28590514, 0.28180913, 0.27748911,
       0.27240109, 0.26838907, 0.26464506, 0.26164105, 0.25742103,
       0.25330501, 0.250057  , 0.24661299, 0.24298497, 0.23935

In [29]:
s = 0
for i in range(P_list.shape[-1]):
    s += np.mean(P_list[:,:,i] * P_list[0,:,i],axis=-1)   
s/=P_list.shape[-1]

In [39]:
name = 'h={}V={}l={}'.format(h, V, length)

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
s = []
for j in range(100):

    P_list = np.load(parentdir+f'/save/dynamics/{name}/P_n=1.0e+04_{j}.npy')
    # new_P_list = np.zeros((P_list.shape[0], P_list.shape[1] * T.shape[0] * T.shape[1], P_list.shape[-1]))

    # for t in range(len(new_P_list)):
    #     new_P_list[t] = f.permute(P_list[t], T, color)
    s.append(0)
    for i in range(P_list.shape[-1]):
        s[j] += np.mean(P_list[:,:,i] * P_list[0,:,i],axis=-1)   
    s[j]/=P_list.shape[-1]
    

FileNotFoundError: [Errno 2] No such file or directory: '/home/keisuke/Documents/dimermaster/save/dynamics/h=1.0V=1.0l=[4, 4]/P_n=1.0e+04_30.npy'

In [38]:
np.mean(s,axis=0)

array([1.        , 0.94220002, 0.89367238, 0.85278195, 0.81783353,
       0.78759265, 0.76063577, 0.73675756, 0.71467223, 0.69511969,
       0.67709836, 0.66038912, 0.64404923, 0.62955479, 0.61589042,
       0.6030336 , 0.5906563 , 0.57926933, 0.56815154, 0.55735927,
       0.54713542, 0.53724334, 0.52767929, 0.51891026, 0.5095603 ,
       0.50082131, 0.49297626, 0.48476938, 0.47668895, 0.46946489,
       0.46177885, 0.45423803, 0.44705278, 0.43985502, 0.43331831,
       0.42679662, 0.42014724, 0.41417393, 0.40786884, 0.40191682,
       0.39557166, 0.38906125, 0.38245067, 0.37661759, 0.37120768,
       0.3657527 , 0.36057442, 0.35535106, 0.35035682, 0.34572441,
       0.34021059, 0.33538036, 0.33002679, 0.32507762, 0.32005459,
       0.31517678, 0.31045172, 0.30556766, 0.3014085 , 0.29711288,
       0.29252304, 0.28798077, 0.28392803, 0.27946715, 0.27512395,
       0.27129407, 0.26745292, 0.26369441, 0.2594977 , 0.25582181,
       0.2526242 , 0.24919872, 0.24534756, 0.24152143, 0.23756

In [19]:
dimer_std[0]/np.sqrt(n_samples_)

array([0.        , 0.00105866, 0.00142574, 0.0016519 , 0.00181669,
       0.00194948, 0.00205127, 0.00213277, 0.00220801, 0.0022732 ,
       0.00232998, 0.00237336, 0.00241301, 0.00245271, 0.00248396,
       0.00251451, 0.00254319, 0.00256785, 0.00259157, 0.00261232,
       0.00263563, 0.00265954, 0.00267409, 0.00269052, 0.00270583,
       0.00272045, 0.00273905, 0.00275149, 0.00276745, 0.00278112,
       0.00279352, 0.00280739, 0.00282142, 0.00283258, 0.00284269,
       0.00285316, 0.00286322, 0.00287274, 0.00288188, 0.00289164,
       0.00289894, 0.00290722, 0.00291519, 0.00292445, 0.00293478,
       0.00293945, 0.00294818, 0.00295388, 0.00296151, 0.00296876,
       0.00297424, 0.00297871, 0.00298434, 0.0029901 , 0.00299472,
       0.00299949, 0.00300466, 0.00300856, 0.00301197, 0.00301617,
       0.00302241, 0.0030258 , 0.00302942, 0.00303376, 0.0030386 ,
       0.00304231, 0.00304569, 0.00304843, 0.00305228, 0.00305485,
       0.00305856, 0.0030628 , 0.00306532, 0.0030676 , 0.00306

In [20]:
dimer_corr[0]

array([1.        , 0.94229769, 0.8925957 , 0.85271411, 0.81851274,
       0.78737149, 0.76107044, 0.73832953, 0.71586863, 0.69516781,
       0.67610704, 0.66084643, 0.64632585, 0.63120525, 0.61886475,
       0.60640426, 0.59432377, 0.58362334, 0.57304292, 0.56354254,
       0.5525821 , 0.54100164, 0.53378135, 0.52546102, 0.5175407 ,
       0.50982039, 0.49975999, 0.49287972, 0.48385935, 0.47595904,
       0.46863875, 0.46027841, 0.45161806, 0.44457778, 0.43807752,
       0.43121725, 0.42449698, 0.41801672, 0.41167647, 0.40477619,
       0.39951598, 0.39345574, 0.3875155 , 0.38047522, 0.3724349 ,
       0.36873475, 0.36169447, 0.35701428, 0.35063403, 0.34445378,
       0.33969359, 0.33575343, 0.33071323, 0.32547302, 0.32119285,
       0.31671267, 0.31177247, 0.30799232, 0.30463219, 0.30045202,
       0.29411176, 0.29061162, 0.28681147, 0.28219129, 0.27693108,
       0.27283091, 0.26903076, 0.26591064, 0.26145046, 0.25843034,
       0.25401016, 0.24884995, 0.24572983, 0.24286971, 0.23984

In [ ]:
dimer_std[0] / np.sqrt(10**5)

In [ ]:
from plotly import graph_objects as go

In [ ]:

data_list = []

for correlation in dimer_corr:
    data_list.append(go.Scatter(
        x = t_list,
        y = correlation * np.sign(correlation[0])
    ))
    

Fig = go.Figure(data = data_list)

In [ ]:
Fig.show()

In [25]:
# import struct
# with open('your_data.dat', 'wb') as fe:  
#     for i in range(dimer_corr.shape[0]):
#         fe.write(struct.pack('{}\n'.format(dimer_corr[i])))
        

In [26]:
file_name = 'vison{}_{:.0e}_T={:.1f}_2.dat'.format(length,n_samples_, t_list[-1])
f.save_corr(file_name, dimer_corr, t_list)

In [68]:
# outfile = open('vison{}_{:.0e}_T={:.1f}.dat'.format(length,n_samples_, t_list[-1]), 'w')
#  # write table header
# for i in range(dimer_corr.shape[0]):
#     outfile.write('#{}\n'.format(i)) 
#     for n in range(dimer_corr.shape[1]):
#         outfile.write('{}\t{:.2f}\t{}\n'.format(n, t_list[n], dimer_corr[i,n]))
#     outfile.write('\n') 
# outfile.close()

In [19]:
tau_list = []
N = 2000
corr1 = []
corr2 = np.zeros((81,N),dtype=np.complex128)
n = -1
import re
with open('K4_4V1.0st.dat') as fe:
    for line in fe:
        
        if re.findall(r'#\d', line):
            i = 0
            n += 1
        else:
            try:
                match = re.findall(r'[\d,\.]+', line)
                tau_list.append(float(match[1]))
                corr1.append(float(match[2]))
                corr2[n,i] = float(match[2]) + 1j*float(match[3])
                i+=1
            except:
                pass

In [67]:
# import re 

# corr = []
# tau = []
# n = -1
# with open(file_name) as fe:
#     for line in fe:
#         if re.findall(r'#\d', line):
#             n += 1
#             corr.append([])
#             tau.append([])
#         else:
#             try:
#                 match = re.findall(r'[\d,\.]+', line)
#                 tau[n].append(float(match[1]))
#                 corr[n].append(float(match[2]))
#             except:
#                 pass

In [65]:
tau = np.array(tau[0])

In [62]:
corr = np.array(corr)

In [64]:
corr.shape

(32, 200)